In [1]:
library(ggplot2)
library(SingleR)
library(dplyr)
library(celldex)
library(RColorBrewer)
source("formating.R")

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
# input = '../../../oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/facs_Bladder_Seurat.h5ad'
input = '../../../oscb/user_storage/Benchmarks/facs-Bladder_1751990659889/results/1391ab45080eed3d0fda22eaa9873b51/facs_Bladder_scanpy.h5ad'
srat <- LoadSeurat(input)
srat

X -> counts



[1] "An error happened when converting AnnData to Seurat, try loading a simple one: the condition has length > 1"


X -> counts



NULL

In [4]:
adata <- LoadAnndata(input)
adata

AnnData object with n_obs × n_vars = 1378 × 2000
    obs: 'orig.ident', 'n_counts', 'n_genes', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'clf_doublet', 'clf_score', 'leiden', 'louvain'
    var: 'name', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'lo

In [3]:
srat[['RNA']]@meta.features

,vst.mean,vst.variance,vst.variance.expected,vst.variance.standardized,vst.variable
,<dbl>,<dbl>,<dbl>,<dbl>,<int>
Cyp2e1,10.108303,26276.128,5506.2117,4.048730,1
Cxcl10,52.645694,172824.496,63205.9315,2.734308,1
Adm,55.160650,242039.182,68397.6301,3.431385,1
Cyp1a1,12.719959,21007.811,7592.5170,2.766910,1
Reg3g,16.383187,47495.256,10644.2449,3.110019,1
Gadd45g,218.296287,1777159.834,785965.6894,2.261116,1
Sprr2f,3.301186,4487.701,1001.2999,4.192271,1
Rbp4,62.066787,182314.488,83660.4194,2.179220,1
Sprr2g,9.603146,16636.223,5106.9485,3.257566,1


In [2]:
sce <- LoadSCE('../../../oscb/user_storage/Benchmarks/facs-Bladder_1750955885784/QC/results/ca3f4d939f5735e1dc7be2cb0a039ac1/facs_Bladder_Bioconductor.rds')
sce

class: SingleCellExperiment 
dim: 16751 1317 
metadata(0):
assays(2): counts logcounts_raw
rownames(16751): 0610005C13Rik 0610007C21Rik ... a l7Rn6
rowData names(5): SYMBOL mean detected discard subset
colnames(1317): A1.B002771.3_39_F.1.1 A1.D041914.3_8_M.1.1 ...
  P9.D042253.3_9_M.1.1 P9.MAA000487.3_10_M.1.1
colData names(40): orig.ident nCount_RNA ... total discard
reducedDimNames(2): PCA TSNE
mainExpName: RNA
altExpNames(0):

In [4]:
sce <- sce[rowSubset(sce)]
sce

class: SingleCellExperiment 
dim: 1948 1317 
metadata(0):
assays(2): counts logcounts_raw
rownames(1948): 0610007C21Rik 0610009D07Rik ... Zfr Zwint
rowData names(5): SYMBOL mean detected discard subset
colnames(1317): A1.B002771.3_39_F.1.1 A1.D041914.3_8_M.1.1 ...
  P9.D042253.3_9_M.1.1 P9.MAA000487.3_10_M.1.1
colData names(40): orig.ident nCount_RNA ... total discard
reducedDimNames(2): PCA TSNE
mainExpName: RNA
altExpNames(0):

In [3]:
rowSubset(sce)

[1] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE
   [13] FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE
   [25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [37] FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE
   [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [61] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE
   [73] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
   [85] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
   [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
  [109] FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE
  [121] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
  [145] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [157] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [169] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [181] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [193] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [205] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [217] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [229] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [241] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [253] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [265] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [277] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [289] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [301] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
  [313]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [325] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [337] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [349] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [361] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [373] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [385] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
  [397] FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [409] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [421] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [433] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [445] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [457] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE
  [469] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE
  [481] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [493] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [505] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE
  [517] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [529] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
  [541] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [553] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [565] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [577] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [589] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
  [6

In [20]:
srat <- CleanSeurat(srat)
srat

An object of class Seurat 
23341 features across 1268 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data

In [12]:
adata <- LoadAnndata(input)
adata

AnnData object with n_obs × n_vars = 1378 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'highly_variable'
    uns: 'leiden', 'louvain', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_umap_3D'
    varm: 'PCs'
    layers: 'raw_counts', 'scale.data'
    obsp: 'connectivities', 'distances'

In [29]:
adata$obs

,orig.ident,n_counts,n_genes,nReads,plate.barcode,mouse.id,tissue,subtissue,FACS.selection,mouse.sex,⋯,pct_counts_mt,pct_counts_rb,pct_counts_hb,percent.plat,RNA_snn_res.0.5,seurat_clusters,doublet_score,doublet_class,leiden,louvain
,<fct>,<dbl>,<int>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>
A1.B000610.3_56_F.1.1,Bladder,610727,2569,610727,B000610,3_56_F,Bladder,,Multiple,F,⋯,0,0,0,0,0,0,0.09696970,Singlet,1,5
A1.B002764.3_38_F.1.1,Bladder,320035,578,320035,B002764,3_38_F,Bladder,,Multiple,F,⋯,0,0,0,0,4,4,0.07878788,Singlet,3,2
A1.B002771.3_39_F.1.1,Bladder,1044981,3998,1044981,B002771,3_39_F,Bladder,,Multiple,F,⋯,0,0,0,0,0,0,0.10909091,Singlet,1,5
A1.D041914.3_8_M.1.1,Bladder,447232,3401,447232,D041914,3_8_M,Bladder,,Multiple,M,⋯,0,0,0,0,0,0,0.09696970,Singlet,0,5
A1.D042253.3_9_M.1.1,Bladder,330249,4439,330249,D042253,3_9_M,Bladder,,Multiple,M,⋯,0,0,0,0,0,0,0.13333333,Singlet,1,0
A1.MAA000487.3_10_M.1.1,Bladder,748761,4471,748761,MAA000487,3_10_M,Bladder,,Multiple,M,⋯,0,0,0,0,2,2,0.13333333,Singlet,0,7
A10.B000610.3_56_F.1.1,Bladder,1486054,4182,1486054,B000610,3_56_F,Bladder,,Multiple,F,⋯,0,0,0,0,1,1,0.20606061,Singlet,2,3
A10.B002764.3_38_F.1.1,Bladder,3783292,3553,3783292,B002764,3_38_F,Bladder,,Multiple,F,⋯,0,0,0,0,1,1,0.23636364,Singlet,2,3
A10.B002771.3_39_F.1.1,Bladder,947995,3634,947995,B002771,3_39_F,Bladder,,Multiple,F,⋯,0,0,0,0,1,1,0.20606061,Singlet,2,3


In [14]:
adata <- adata[adata$obs['doublet_class']=='Singlet']
adata

View of AnnData object with n_obs × n_vars = 1268 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'nReads', 'plate.barcode', 'mouse.id', 'tissue', 'subtissue', 'FACS.selection', 'mouse.sex', 'percent.ercc', 'free_annotation', 'cell_ontology_class', 'percent.ribo', 'res.0.4', 'cluster.ids', 'cell_ontology_id', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'percent.plat', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_score', 'doublet_class', 'leiden', 'louvain'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'highly_variable'
    uns: 'leiden', 'louvain', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_umap_3D'
    varm: 'PCs'
    layers: 'raw_counts', 'scale.data'
    obsp: 'connectivities', 'distances'

In [17]:
if('outlier' %in% names(adata$obs) && 'mt_outlier' %in% names(adata$obs)){
        adata <- adata[adata$obs['outlier']=='False']
        adata <- adata[adata$obs['mt_outlier']=='False']}

In [16]:
'outlier' %in% names(adata$obs) && 'mt_outlier' %in% names(adata$obs)

[1] FALSE

In [16]:
srat <- AnndataToSeurat(adata, complete = TRUE)
srat

X -> counts

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from raw_counts_ to rawcounts_”
Adding AnnData layers to Seurat assays

Adding AnnData layers to Seurat assays

Adding AnnData embeddings to Seurat assays

Adding AnnData embeddings to Seurat assays

Adding AnnData embeddings to Seurat assays

Adding AnnData embeddings to Seurat assays



An object of class Seurat 
46682 features across 1378 samples within 2 assays 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: raw_counts
 4 dimensional reductions calculated: pca, tsne, umap, umap_3D

In [17]:
names(srat@meta.data)

[1] "orig.ident"                    "nCount_RNA"                   
 [3] "nFeature_RNA"                  "nCounts_RNA"                  
 [5] "nFeatures_RNA"                 "nReads"                       
 [7] "plate.barcode"                 "mouse.id"                     
 [9] "tissue"                        "subtissue"                    
[11] "FACS.selection"                "mouse.sex"                    
[13] "percent.ercc"                  "free_annotation"              
[15] "cell_ontology_class"           "percent.ribo"                 
[17] "res.0.4"                       "cluster.ids"                  
[19] "cell_ontology_id"              "percent.mt"                   
[21] "percent.rb"                    "percent.hb"                   
[23] "percent.plat"                  "RNA_snn_res.0.5"              
[25] "seurat_clusters"               "doublet_score"                
[27] "doublet_class"                 "leiden"                       
[29] "louvain"                       "nFeatures_RNA_by_counts"      
[31] "log1p_nFeatures_RNA_by_counts" "total_counts"                 
[33] "log1p_total_counts"            "pct_counts_in_top_20_genes"   
[35] "total_counts_mt"               "log1p_total_counts_mt"        
[37] "total_counts_ribo"             "log1p_total_counts_ribo"      
[39] "pct_counts_ribo"               "total_counts_hb"              
[41] "log1p_total_counts_hb"         "outlier"                      
[43] "mt_outlier"                    "doublet_scores"               
[45] "predicted_doublets"

In [6]:
srat <- subset(srat, subset = discard == FALSE)
srat

An object of class Seurat 
16751 features across 1317 samples within 1 assay 
Active assay: RNA (16751 features, 0 variable features)
 2 layers present: counts, data

In [8]:
'discard' %in% names(srat[['RNA']]@meta.features)

[1] TRUE

In [16]:
srat@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,nCounts_RNA,nFeatures_RNA,nReads,plate.barcode,mouse.id,tissue,subtissue,⋯,log1p_total_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,outlier,mt_outlier,doublet_scores,predicted_doublets
,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<dbl>,<lgl>
A1.B000610.3_56_F.1.1,Bladder,184259,406,610727,2569,610727,B000610,3_56_F,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.03057134,FALSE
A1.B002764.3_38_F.1.1,Bladder,301835,115,320035,578,320035,B002764,3_38_F,Bladder,,⋯,0,0,0,0,0,0,TRUE,FALSE,0.21470823,FALSE
A1.B002771.3_39_F.1.1,Bladder,349384,545,1044981,3998,1044981,B002771,3_39_F,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.06290644,FALSE
A1.D041914.3_8_M.1.1,Bladder,133001,473,447232,3401,447232,D041914,3_8_M,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.02486694,FALSE
A1.D042253.3_9_M.1.1,Bladder,145722,582,330249,4439,330249,D042253,3_9_M,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.08281356,FALSE
A1.MAA000487.3_10_M.1.1,Bladder,316135,605,748761,4471,748761,MAA000487,3_10_M,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.03492313,FALSE
A10.B000610.3_56_F.1.1,Bladder,473566,403,1486054,4182,1486054,B000610,3_56_F,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.05886360,FALSE
A10.B002764.3_38_F.1.1,Bladder,988155,318,3783292,3553,3783292,B002764,3_38_F,Bladder,,⋯,0,0,0,0,0,0,TRUE,FALSE,0.06290644,FALSE
A10.B002771.3_39_F.1.1,Bladder,304801,345,947995,3634,947995,B002771,3_39_F,Bladder,,⋯,0,0,0,0,0,0,FALSE,FALSE,0.06290644,FALSE


In [19]:
adata$var

,vst.mean,vst.variance,vst.variance.expected,vst.variance.standardized,highly_variable
,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
0610005C13Rik,3.120464e-02,8.131142e-01,5.298699e-01,1.0455663,FALSE
0610007C21Rik,1.503345e+02,6.628982e+04,7.510799e+04,0.8825934,FALSE
0610007L01Rik,5.860740e+01,3.403284e+04,1.825166e+04,1.7938585,TRUE
0610007N19Rik,3.363570e+01,6.287461e+03,8.654142e+03,0.7265262,FALSE
0610007P08Rik,1.565893e+01,1.039076e+04,3.189147e+03,1.7089364,TRUE
0610007P14Rik,3.329100e+01,6.613174e+03,8.537044e+03,0.7746445,FALSE
0610007P22Rik,2.035486e+01,4.417849e+03,4.479372e+03,0.9862653,FALSE
0610008F07Rik,7.256894e-04,7.256894e-04,6.806306e-04,1.0014999,FALSE
0610009B14Rik,3.410740e-01,7.452265e+01,2.067955e+01,1.1714712,FALSE


In [20]:
srat[['RNA']]@meta.features <- adata$var

In [8]:
VariableFeatures(srat)

[1] "Cyp2e1"        "Cxcl10"        "Adm"           "Cyp1a1"       
   [5] "Reg3g"         "Gadd45g"       "Sprr2f"        "Rbp4"         
   [9] "Sprr2g"        "Car3"          "Has1"          "Spon2"        
  [13] "Inmt"          "Tnfaip6"       "Hk2"           "Cxcl1"        
  [17] "8430408G22Rik" "Upk3a"         "Pi16"          "Hmox1"        
  [21] "Gem"           "Top2a"         "Ccl2"          "Akr1b8"       
  [25] "E330033B04Rik" "Gm2a"          "Myoc"          "Ptx3"         
  [29] "Alg8"          "Dpt"           "Sostdc1"       "Ifit1"        
  [33] "Ifit3"         "Krt13"         "Csrp1"         "Fmod"         
  [37] "Ccnb2"         "Acta2"         "Rrad"          "Bglap2"       
  [41] "Ovol1"         "Gas7"          "Nupr1"         "Hspa1a"       
  [45] "Trex2"         "C1qtnf3"       "Slpi"          "Psca"         
  [49] "4921531C22Rik" "Mki67"         "Ccnb1"         "Bdnf"         
  [53] "Cdr2l"         "Ly6k"          "Krt4"          "Mfap5"        
  [57] "Degs2"         "Cpxm1"         "Arc"           "Il6"          
  [61] "Notum"         "Gpx3"          "Wnt10a"        "Ccrl2"        
  [65] "Kcns3"         "Gjb6"          "Ube2c"         "Gdf15"        
  [69] "Cdc20"         "3632451O06Rik" "Il33"          "Avpr1a"       
  [73] "Prss23"        "Slc19a1"       "Rasl11a"       "Kazald1"      
  [77] "Mn1"           "Clcf1"         "Btbd3"         "Krt16"        
  [81] "Krt20"         "Ccl7"          "Cd34"          "Ncs1"         
  [85] "Slc46a1"       "Sertad3"       "H2-Ab1"        "Cyp2f2"       
  [89] "Nampt"         "Clec3b"        "Fbp1"          "Ngb"          
  [93] "Bdkrb1"        "Opn3"          "Tmem146"       "Il1rn"        
  [97] "Hn1"           "Gsn"           "Slc2a6"        "Larp6"        
 [101] "Ereg"          "Plau"          "B4galt7"       "Rrm2"         
 [105] "Isl1"          "Prc1"          "Pcolce2"       "D0H4S114"     
 [109] "2210012G02Rik" "Irf1"          "Rasd1"         "Tgtp2"        
 [113] "Tmem100"       "Gypc"          "Ccdc80"        "Racgap1"      
 [117] "Nfkbia"        "Nrg1"          "Ggt6"          "Hpcal1"       
 [121] "Cyr61"         "Krt14"         "Gbp2"          "Clu"          
 [125] "Banp"          "Cldn4"         "Lum"           "Alkbh5"       
 [129] "Ifi47"         "Apoe"          "Birc5"         "Bambi"        
 [133] "Vcam1"         "Gstm4"         "Rnf25"         "Taok2"        
 [137] "Areg"          "Slc25a33"      "Tmem191c"      "Sfrp4"        
 [141] "Vwa1"          "Cars2"         "Abcd4"         "Slc39a1"      
 [145] "Pwp1"          "Timp3"         "Tbc1d4"        "Tnfsf9"       
 [149] "Cyp2c44"       "Upk1a"         "Sod1"          "Dhodh"        
 [153] "Tmprss13"      "Bend4"         "Itpripl1"      "Cdk1"         
 [157] "Tor2a"         "Rap2b"         "Aaas"          "Slc29a1"      
 [161] "Stk16"         "Fbxl8"         "Cxcl2"         "Mpzl1"        
 [165] "Pitx1"         "Rln1"          "Snupn"         "Chadl"        
 [169] "Sod3"          "Mtrf1"         "Pip5k1c"       "Lgals3"       
 [173] "Cdc42ep3"      "Tapbpl"        "Mus81"         "Epgn"         
 [177] "Chst8"         "Plaur"         "Ccna2"         "Srxn1"        
 [181] "Serpine2"      "Cd14"          "Cd200"         "Pnck"         
 [185] "Cldn23"        "Fitm2"         "Prr3"          "Akr1b7"       
 [189] "Tln2"          "Cdca3"         "S1pr2"         "AW209491"     
 [193] "Wdr8"          "2310043J07Rik" "Ccdc6"         "Abcg1"        
 [197] "Dnase2a"       "St6galnac4"    "Dtwd1"         "Aplf"         
 [201] "Ckap2l"        "Ppm1h"         "Cd40"          "Itm2a"        
 [205] "Smox"          "Mum1"          "Grpel2"        "Chpf"         
 [209] "Esam"          "Rgs2"          "Mfap4"         "Yipf2"        
 [213] "Cgn"           "Gjb4"          "Cyb561d2"      "Snx33"        
 [217] "Fam109b"       "Whsc2"         "Polr1c"        "Fam83d"       
 [221] "Chac1"         "Msto1"         "Zdhhc12"       "Pdgfrl"       
 [225] "Qpctl"         "Yar

In [9]:
srat <- srat[VariableFeatures(object = srat), ]
srat

An object of class Seurat 
2000 features across 1378 samples within 1 assay 
Active assay: RNA (2000 features, 2000 variable features)
 2 layers present: counts, data

In [10]:
srat <- subset(srat, subset = doublet_class == 'Singlet')
srat

An object of class Seurat 
2000 features across 1268 samples within 1 assay 
Active assay: RNA (2000 features, 2000 variable features)
 2 layers present: counts, data

In [14]:
srat <- subset(srat, meta.features = srat[['RNA']]@meta.features['discard'==FALSE])
srat

Warning message:
“The following arguments are not used: meta.features”


An object of class Seurat 
16751 features across 1317 samples within 1 assay 
Active assay: RNA (16751 features, 0 variable features)
 2 layers present: counts, data

In [12]:
srat[['RNA']]@meta.features['discard'==FALSE]

Warning message in rbind(parts$upper, chars$ellip_v, parts$lower, deparse.level = 0L):
“number of columns of result is not a multiple of vector length (arg 2)”
Warning message in rbind(parts$upper, chars$ellip_v, parts$lower, deparse.level = 0L):
“number of columns of result is not a multiple of vector length (arg 2)”
Warning message in rbind(parts$upper, chars$ellip_v, parts$lower, deparse.level = 0L):
“number of columns of result is not a multiple of vector length (arg 2)”
Warning message in rbind(parts$upper, chars$ellip_v, parts$lower, deparse.level = 0L):
“number of columns of result is not a multiple of vector length (arg 2)”


0610005C13Rik
0610007C21Rik
0610007L01Rik
0610007N19Rik
0610007P08Rik
0610007P14Rik
0610007P22Rik
0610009B14Rik
0610009B22Rik
0610009D07Rik
0610009L18Rik


In [27]:
srat <- subset(srat[['RNA']], features = vst.variable == TRUE)
srat

ERROR: Error in features %||% rownames(x = x): object 'vst.variable' not found


In [6]:
srat <- FindVariableFeatures(srat, selection.method = "vst")

In [4]:
srat[['RNA']]@counts

  [[ suppressing 32 column names ‘10X_P4_3_AAAGTAGAGATGCCAG’, ‘10X_P4_3_AACCGCGTCCAACCAA’, ‘10X_P4_3_AACTCCCGTCGGGTCT’ ... ]]

  [[ suppressing 32 column names ‘10X_P4_3_AAAGTAGAGATGCCAG’, ‘10X_P4_3_AACCGCGTCCAACCAA’, ‘10X_P4_3_AACTCCCGTCGGGTCT’ ... ]]

  [[ suppressing 32 column names ‘10X_P4_3_AAAGTAGAGATGCCAG’, ‘10X_P4_3_AACCGCGTCCAACCAA’, ‘10X_P4_3_AACTCCCGTCGGGTCT’ ... ]]



23341 x 2500 sparse Matrix of class "dgCMatrix"
                                                                                          
Xkr4           .   .  .   .   .   .   .  .  .   . .  .  .  .  .  .  .   .   .   .  .  .  .
Rp1            .   .  .   .   .   .   .  .  .   . .  .  .  .  .  .  .   .   .   .  .  .  .
Sox17          .   .  .   .   .   .   .  .  .   . .  .  .  .  .  .  .   .   .   .  .  .  .
Mrpl15         1   .  .   .   .   2   .  .  .   . .  .  .  .  .  .  1   .   1   1  1  .  .
Lypla1         .   2  1   2   .   3   1  .  .   1 .  .  .  1  .  .  .   .   .   1  .  .  .
Tcea1          .   3  1   .   1   1   1  .  1   . .  1  .  1  .  .  1   .   .   2  1  1  .
Rgs20          .   .  .   .   .   .   .  .  .   . .  .  .  .  .  .  .   .   .   .  .  .  .
Atp6v1h        .   .  2   1   1   .   1  .  1   . .  .  .  .  .  .  .   .   .   1  .  1  .
Oprk1          .   .  .   .   .   .   .  .  .   . .  .  .  .  .  .  .   .   .   .  .  .  .
Npbwr1         .   .  .   .   .   .   .  .

In [ ]:
source("../tools/qc/seurat_qc.R")
path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"
results <- LoadSeuratMetaData(path)
results$srat

In [ ]:
qc_results <- RunSeuratQC(results$srat, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat')
qc_results

In [ ]:
# library("here")
here()

In [ ]:
getwd()

In [ ]:
packageVersion("Seurat")

In [ ]:
current_path <- getwd()
print(current_path)

In [ ]:
here::here('api/tools/qc/scrublet_calls.tsv')

In [ ]:
source("../tools/qc/seurat_qc.R")
# path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# adata_path = '/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'
# path="/ps/ai-ready/data/tung/umi.h5Seurat"
path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
adata_path = 'ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'
# adata_path = '/ps/ai-ready/data/tung/umi.h5ad'

results <- RunSeuratQC(path, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', adata_path=adata_path, assay='originalexp')
results

In [ ]:
print(results)

In [ ]:
source("../tools/qc/seurat_qc.R")
# path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"
# adata_path = '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad'
path="/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
adata_path = '/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad'

results <- RunSeuratQC(path, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', unique_id='Seurat test', adata_path=adata_path)
results

In [ ]:
# srat <- LoadSeurat('/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.Robj')
# srat <- LoadSeurat("/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat")
srat <- LoadSeurat("/ps/ai-ready/data/tung/umi.h5Seurat")
# srat@reductions <- NULL
srat

In [ ]:
DefaultAssay(srat) <- "originalexp"
srat

In [ ]:
# remotes::install_github('chris-mcginnis-ucsf/DoubletFinder')

In [ ]:
srat <- NormalizeData(srat)
srat <- FindVariableFeatures(srat, selection.method="vst", nfeatures=2000)
srat <- ScaleData(srat)
srat <- RunPCA(srat)
srat <- RunUMAP(srat, dims=1:10)

In [ ]:
srat

In [ ]:
srat@meta.data

In [ ]:
library(DoubletFinder)
set.seed(123)
sweep.res.list <- paramSweep(srat, PCs=1:10, sct=FALSE)
sweep.stats <- summarizeSweep(sweep.res.list, GT=FALSE)
bcmvn <- find.pK(sweep.stats)

In [ ]:
packageVersion("DoubletFinder")

In [ ]:
nExp_poi <- round(0.075*nrow(srat@meta.data)) 

In [ ]:
srat <- doubletFinder(srat, PCs = 1:10, pN = 0.25, pK = 0.09, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)

In [ ]:
library(stringr)
colnames(srat@meta.data)[str_starts(colnames(srat@meta.data),"pANN_")] <- "Double_score"
colnames(srat@meta.data)[str_starts(colnames(srat@meta.data),"DF.classifications_")] <- "Is_Double"

In [ ]:
srat <- FindNeighbors(srat, dims=1:10)
srat <- FindClusters(srat, resolution=0.5)

In [ ]:
srat@meta.data

In [ ]:
sce <- as.SingleCellExperiment(srat)
sce

In [ ]:
# BiocManager::install("plger/scDblfinder")
# install.packages("irlba",type = "source")
remotes::install_github("Moonerss/scrubletR")

In [ ]:
scrublet_R <- function(seurat_obj, python_home = Sys.which("python3"),
                       return_results_only = FALSE, min_counts=2,
                       min_cells=3, expected_doublet_rate=0.06,
                       min_gene_variability_pctl=85,
                       n_prin_comps=50, sim_doublet_ratio=2, n_neighbors=NULL) {
  ## use the python
  reticulate::use_python(python_home)

  ## test whether have the modules
  if (!reticulate::py_module_available("scrublet")) {
    stop("python module scrublet does not seem to be installed; - try running 'py_config()'")
  }

  ## source .py file
  reticulate::source_python(paste(system.file(package = "scrubletR"), "scrublet.py", sep = "/"))

  ## prepare the data
  X <- as(Matrix::t(seurat_obj@assays$RNA@counts), "TsparseMatrix")
  i <- as.integer(X@i)
  j <- as.integer(X@j)
  val <- X@x
  dim <- as.integer(X@Dim)
  if(is.null(n_neighbors)){
    n_neighbors<-round(0.5*sqrt(nrow(X)))
  }

  ## do the scrublet analysis
  scrublet_py_args<-c(list(i=i, j=j, val=val, dim=dim,
                           expected_doublet_rate=expected_doublet_rate, min_counts=min_counts,
                           min_cells=min_cells, min_gene_variability_pctl=min_gene_variability_pctl, n_prin_comps=n_prin_comps,
                           sim_doublet_ratio=sim_doublet_ratio, n_neighbors=n_neighbors))
  scrublet_res <- do.call(scrublet_py, scrublet_py_args)
  names(scrublet_res)<-c("doublet_scores", "predicted_doublets")

  ## get the result
  if (return_results_only) {
    return(scrublet_res)
  } else {
    seurat_obj[["doublet_scores"]] <- scrublet_res$doublet_scores
    seurat_obj[["predicted_doublets"]] <- scrublet_res$predicted_doublets
    return(seurat_obj)
  }
}

In [ ]:
# library(scrubletR)
res <- scrublet_R(srat)
res

In [ ]:
sce <- runScrublet(sce)
sce

In [ ]:
sce <- scDblFinder(sce, nfeatures=2000, dims=30, iter=5)

In [ ]:
doublet_annnotation <- AnnotateDroplet(srat[["RNA"]]@counts)

In [ ]:
doublet_annnotation

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
path="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"

srat <- LoadSeurat(path)

srat
srat@version

In [ ]:
Project(srat)

In [ ]:
obj <- readRDS("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj")
obj

In [ ]:
srat_v2 <- get(load(path))
srat_v2

In [ ]:
class(srat_v2) == 'seurat'

In [ ]:
srat_v2@meta.data

In [ ]:
srat[['RNA']]@counts

In [ ]:
v4_obj <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data)
v4_obj

In [ ]:
SaveH5Seurat(v4_obj, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
Cells(v4_obj)

In [ ]:
v4_obj@meta.data

In [ ]:
v4_obj[['RNA']]@var.features

In [ ]:
srat@reductions$pca@misc <- list()
srat@reductions$tsne@misc <- list()
# srat@reductions$umap@misc <- list()

In [ ]:
SaveH5Seurat(srat, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
saveRDS(srat, "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")

In [ ]:
srat1 <- readRDS(file="/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")
srat1

In [ ]:
as.h5Seurat(srat1, '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite = TRUE, verbose = TRUE)

In [ ]:
SaveH5Seurat(srat1, filename='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5seurat', overwrite=TRUE)

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds', dest = "h5ad" , overwrite = TRUE)

In [ ]:
class(srat1) == 'Seurat'

In [ ]:
v4_obj@version

In [ ]:
srat2 <- readRDS(file="/ps/ai-ready/data/tung/tung.rds")
srat2

In [ ]:
class(srat2)

In [ ]:
# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(srat, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
plot1 <- FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

In [ ]:
srat <- subset(srat, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))

In [ ]:
srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)

In [ ]:
DimHeatmap(srat, dims = c(8, 10))

In [ ]:
srat <- CellCycleScoring(srat, g2m.features = cc.genes$g2m.genes, s.features = cc.genes$s.genes)

In [ ]:
# Read in the expression matrix The first row is a header row, the first column is rownames
exp.mat <- read.table(file = "../tools/qc/nestorawa_forcellcycle_expressionMatrix.txt", header = TRUE,
    as.is = TRUE, row.names = 1)

# A list of cell cycle markers, from Tirosh et al, 2015, is loaded with Seurat.  We can
# segregate this list into markers of G2/M phase and markers of S phase
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes

In [ ]:
srat <- CellCycleScoring(srat, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)
# view cell cycle scores and phase assignments
head(srat[[]])

In [ ]:
VlnPlot(srat, features = c("S.Score", "G2M.Score"), group.by = "orig.ident",
    ncol = 4, pt.size = 0.1)

In [ ]:
srat <- ScaleData(srat, vars.to.regress = c("S.Score", "G2M.Score"), features = rownames(srat))
srat

In [ ]:
remotes::install_github('chris-mcginnis-ucsf/DoubletFinder')

In [ ]:
# define the expected number of doublet cellscells.
nExp <- round(ncol(srat) * 0.04)  # expect 4% doublets
srat <- doubletFinder_v3(srat, pN = 0.25, pK = 0.09, nExp = nExp, PCs = 1:10)

In [ ]:
rm(srat)
gc()